In [1]:
## SETUP (OPTIONAL - USE IF REQUIRED)
## Bracketed Paste is disabled to prevent characters after output
## Example: 
## $ echo 'Hii'
## | Hi?2004l
# bind 'set enable-bracketed-paste off'

In [2]:
## TEST:  Make sure simple prompt used (e.g., no escapes that might contaminate output)
## PS1="$ "
## TODO: PS1="> "

## NOTE: The Jupyter bash kernel requires that PS1 not be modified as they customize it. 
echo $PS1

PROMPT_KPEYTACPMGNW\[\]>


In [3]:
## BAD (SUPER BAD)
# # Delete all aliases and function
# # TODO: Instead start from pristine environment
# unalias -a
# alias | wc -l
# for f in $(typeset -f | egrep '^\w+'); do unset -f $f; done
# typeset -f | egrep '^\w+' | wc -l

In [4]:
# Global Setup
alias testuser="sed -r "s/"$USER"+/userxf333/g""

In [5]:
# Setting a temp directory for tests
TMP=/tmp/test-cond-env

# OLD
# ## NOTE: Source it directly from the ./tests directory.
# BIN_DIR=$PWD/..
# ## You will need to run jupyter from that directory.
# ## source $TEMP_BIN/_dir-aliases.bash
# source _dir-aliases.bash
# alias | wc -l

In [6]:
## NOTE: For reproducability, the directory name needs to be fixed
## In place of $$, use a psuedo random number (e,g., 1710)
## *** All output from one run to the next needs to be the same ***

## temp_dir=$TMP/test-$$
temp_dir=$TMP/test-1710

## OLD
# mkdir -p "$temp_dir"
# # TODO: /bin/rm -rvf "$temp_dir"
# cd "$temp_dir"

command mkdir -p "$temp_dir"
# # TODO: /bin/rm -rvf "$temp_dir"
command cd "$temp_dir"

## OLD: For debugging
# pwd

#ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"

In [7]:
# Count aliases proper
alias | wc -l

440


In [8]:
# Count functions
typeset -f | egrep '^\w+' | wc -l

471


In [9]:
## BAD
# source $BIN_DIR/tomohara-aliases.bash

In [10]:
# SETTING A ENVIRONMENT VARIABLE NAMED COND_ENV_TEST_TEMP
cond-setenv COND_ENV_TEST_TEMP tmp/cond-env-test-temp

In [11]:
# PROOF THAT THE ENVIRONMENT VARIABLE IS CREATED
echo $COND_ENV_TEST_TEMP

tmp/cond-env-test-temp


In [12]:
# USE OF ENV VARIABLE TO CREATE A SUB-DIRECTORY
rm -rf ./* > /dev/null
mkdir -p $COND_ENV_TEST_TEMP
ls -l | awk '!($6=$7=$8="")' | testuser

total 4      
drwxr-xr-x 3 userxf333 userxf333 4096    tmp


In [13]:
# THE OUTPUT OF printenv IS STORED IN temp-env-1.tmp CREATED IN THE DIRECTORY SPECIFIED BY $COND_ENV_TEST_TEMP
## OLD: For debug
# pwd
printenv >> $COND_ENV_TEST_TEMP/temp-env-1.tmp

In [14]:
cat $COND_ENV_TEST_TEMP/temp-env-1.tmp | grep TOM_BIN | testuser

TOM_BIN=/home/userxf333/tomasProject/shell-scripts


In [15]:
echo "Done!"

Done!
